In [17]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img, ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Input

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
from tqdm.keras import TqdmCallback

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2



In [3]:
data_dir = "../cropped"

In [4]:

def create_dataframe(dir):
    image_paths = []
    card_type = []

    for cardtype in os.listdir(dir):
        cardtype_path = os.path.join(dir, cardtype)

        if os.path.isdir(cardtype_path):
            for filename in os.listdir(cardtype_path):
                if filename.endswith('jpg'):
                    image_paths.append(os.path.join(dir, cardtype, filename))
                    # splitting by shading
                    card_type.append(cardtype[3:4])
    return image_paths, card_type

In [5]:
data = pd.DataFrame()
data['images'], data['card type'] = create_dataframe(data_dir)

In [6]:
def reshape_images(images):
    reshaped = []
    for image in tqdm(images):
        img = load_img(image)
        img = np.array(img)
        reshaped.append(img)
    reshaped = np.array(reshaped)
    reshaped = reshaped.reshape(len(reshaped), 200, 150, 3)

    return reshaped

In [7]:
data_reshaped = reshape_images(data['images'])

x = data_reshaped / 255.0

  0%|          | 0/2307 [00:00<?, ?it/s]

In [10]:
le = LabelEncoder()

le.fit(data['card type'])


y = le.transform(data['card type'])
y = to_categorical(y, num_classes=3)

In [15]:
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (2307, 200, 150, 3)
Shape of y: (2307, 3)


In [23]:
model = Sequential()

model.add(Input(shape=(200, 150, 3)))
model.add(Conv2D(32, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

In [24]:
model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(3, activation= "softmax"))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [25]:
checkpoint = ModelCheckpoint("../models/shading.keras", monitor = "val_accuracy", verbose = 1, save_best_only = True, mode = 'max')

early_stopping = EarlyStopping(monitor = "val_loss", min_delta = 0.001, patience = 5, verbose = 1, restore_best_weights = True)

reduced_learning_rate = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3, verbose = 1, min_delta = 0.0001)


callbacks_list = [checkpoint, early_stopping, reduced_learning_rate]

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [27]:
history = model.fit(x = x_train, y = y_train, batch_size = 32, epochs = 50, validation_data = (x_test, y_test), callbacks = callbacks_list)

Epoch 1/50
58/58 [==============================] - ETA: 0s - loss: 0.9351 - accuracy: 0.4808
Epoch 1: val_accuracy improved from -inf to 0.62121, saving model to models/42.2.keras
58/58 [==============================] - 27s 451ms/step - loss: 0.9351 - accuracy: 0.4808 - val_loss: 0.7071 - val_accuracy: 0.6212 - lr: 0.0010
Epoch 2/50
58/58 [==============================] - ETA: 0s - loss: 0.5332 - accuracy: 0.6770
Epoch 2: val_accuracy improved from 0.62121 to 0.81169, saving model to models/42.2.keras
58/58 [==============================] - 26s 441ms/step - loss: 0.5332 - accuracy: 0.6770 - val_loss: 0.4920 - val_accuracy: 0.8117 - lr: 0.0010
Epoch 3/50
58/58 [==============================] - ETA: 0s - loss: 0.5327 - accuracy: 0.6976
Epoch 3: val_accuracy did not improve from 0.81169
58/58 [==============================] - 25s 433ms/step - loss: 0.5327 - accuracy: 0.6976 - val_loss: 0.4803 - val_accuracy: 0.6883 - lr: 0.0010
Epoch 4/50
58/58 [==============================] - ETA

In [29]:
from keras.models import load_model

In [57]:
model = load_model('../models/shading.keras')

In [58]:
def prep(image):
    img = load_img(image)
    feature = np.array(img)
    feature = feature.reshape(1, 200, 150, 3)
    return feature / 255.0

In [59]:
def predict_shading(imagepath):
    image = prep(imagepath)
    pred = model.predict(image).argmax()

    if pred == 0:
        return "Empty", 0
    if pred == 1: 
        return "Stripe", 1
    else:
        return "Solid", 2


In [63]:
imagepath = '../cropped/1312/10.jpg'

pred = predict_shading(imagepath)
print("prediction: ", pred[0])

loaded_image = cv2.imread(imagepath)
cv2.imshow('image ', loaded_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 26ms/step
prediction:  Stripe
